## Import modules:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns

## Function definitions: 

In [2]:
def essentiality_calls(df, gene_id):
# Return the columns in which a given gene is called essential:
    cols = df.columns[2:]
    col_list = []
    for col in cols:
        if df[df['Rv_ID']==gene_id][col].values[0] == 1:
            col_list.append(col)
            
    return col_list

## Load datasets:

Screen condition descriptions:

In [3]:
dir_data = '../../data'
column_file = os.path.join(dir_data, 'column_descriptors.csv')
df_column = pd.read_csv(column_file)

TnSeq matrix:

In [4]:
tn_file = os.path.join(dir_data, 'Tn_library_DB.xlsx')
df_tn = pd.read_excel(tn_file)
df_tn.shape

(3990, 60)

## Read in list of genes to query:

File names: 

In [8]:
gene_list_file = 'Mtb_all_redox_genes_wUK_TOP.xlsx'
xls_name = gene_list_file.split('.')[0] + '_TnMat.xlsx'

# Create a Pandas Excel output writer:
excel_output = os.path.join(dir_data, 'other_data', xls_name)
writer = pd.ExcelWriter(excel_output, engine='xlsxwriter')

Using excel file: 

In [ ]:
df_gene_list = pd.read_excel(os.path.join(dir_data, 'other_data', gene_list_file))
gene_list = df_gene_list.Rv_ID.values
gene_list

Using python list:

In [10]:
gene_list = ['Rv0733', 'Rv1565c']

# Create a Pandas Excel output writer:
xls_name = 'gene_kyu.xlsx'
excel_output = os.path.join(dir_data, 'other_data', xls_name)
writer = pd.ExcelWriter(excel_output, engine='xlsxwriter')

## Query genes:

output results to excel sheets:

In [11]:
save_to_xls = True

In [12]:
essential_dict = {}
for gene in gene_list:
    hits = essentiality_calls(df_tn, gene)
    df_temp = df_column[df_column.column_ID.isin(hits)]
    essential_dict[gene] = hits
    if save_to_xls:
        df_temp.to_excel(writer, sheet_name=gene, index = False)

if save_to_xls:
    writer.save()

### This is a simple format in which to save the dataframe:

In [ ]:
essential_dict

In [60]:
df_by_genes = pd.DataFrame()
df_by_genes['Rv_ID'] = essential_dict.keys()
df_by_genes['TnSeq_calls'] = essential_dict.values()
df_by_genes['num_TnSeq'] = [len(TnSeq) for TnSeq in df_by_genes.TnSeq_calls.values]
df_by_genes.sort_values(by='num_TnSeq', ascending=False, inplace=True)

df_by_genes = df_by_genes[['Rv_ID', 'num_TnSeq', 'TnSeq_calls']]

In [63]:
xls_out_name = 'redox_TnSeq_by_genes.xlsx'
excel_output = os.path.join(dir_data, 'other_data', xls_out_name)

df_by_genes.to_excel(excel_output, index = False)


## For a list of genes, make an inverse mapping of TnSeq screens --> to --> genes

In [48]:
TnSeq_screens = []
for val in essential_dict.values():
    TnSeq_screens += val
TnSeq_screens = list(set(TnSeq_screens))
TnSeq_screens.sort()

In [58]:
TnSeq_dict = {}
for TnSeq in TnSeq_screens:
    genes_in_TnSeq = [ gene for gene in essential_dict.keys() if TnSeq in essential_dict[gene]]
    genes_in_TnSeq.sort()
    TnSeq_dict[TnSeq] = genes_in_TnSeq
    
df_TnSeq = pd.DataFrame()
df_TnSeq['TnSeq_screen'] = TnSeq_dict.keys()
df_TnSeq['genes'] = TnSeq_dict.values()
df_TnSeq['num_genes'] = [len(gs) for gs in df_TnSeq.genes.values]
df_TnSeq.sort_values(by='num_genes', ascending=False, inplace=True)

df_TnSeq = df_TnSeq[['TnSeq_screen', 'num_genes', 'genes']]


In [56]:
xls_out_name = 'redox_genes_by_TnSeq.xlsx'
excel_output = os.path.join(dir_data, 'other_data', xls_out_name)

df_TnSeq.to_excel(excel_output, index = False)
